In [1]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import ccxt
import itertools
import ipywidgets
from datetime import datetime, timedelta
from numba import njit


In [37]:
seed = 42
symbols = ['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'ADAUSDT', 'LTCUSDT']
start_date = datetime(2020, 3, 24)
end_date = datetime(2023, 1, 19)
time_delta = end_date - start_date
window_len = timedelta(days=180)
window_count = 400
exit_types = ['SL', 'TS', 'TP', 'Random', 'Holding']
step = 0.01  # in %
stops = np.arange(step, 1 + step, step)

# vbt.settings.layout['template'] = vbt.settings.dark_template

vbt.settings.plotting["layout"]["template"] = "vbt_dark"
vbt.settings.portfolio['freq'] = '1d'
vbt.settings.portfolio['init_cash'] = 100.  # in $
vbt.settings.portfolio['fees'] = 0.0025  # in %
vbt.settings.portfolio['slippage'] = 0.0025  # in %


In [3]:
print(pd.Series({
    'Start date': start_date,
    'End date': end_date,
    'Time period (days)': time_delta.days,
    'Assets': len(symbols),
    'Window length': window_len,
    'Windows': window_count,
    'Exit types': len(exit_types),
    'Stop values': len(stops),
    'Tests per asset': window_count * len(stops) * len(exit_types),
    'Tests per window': len(symbols) * len(stops) * len(exit_types),
    'Tests per exit type': len(symbols) * window_count * len(stops),
    'Tests per stop type and value': len(symbols) * window_count,
    'Tests total': len(symbols) * window_count * len(stops) * len(exit_types)
}))


In [49]:
cols = ['Open', 'Low', 'High', 'Close', 'Volume']
data = vbt.CCXTData.download(
    symbols,
    start=start_date,
    end=end_date,
    timeframe='1d',
    exchange='bybit'
    )
data.save('stop_sigs')
data = vbt.CCXTData.load('stop_sigs')
ohlcv_by_symbol = data.data


In [50]:
print(ohlcv_by_symbol.keys())
print(ohlcv_by_symbol['BTCUSDT'].shape)
ohlcv_by_symbol['BTCUSDT'].vbt.ohlcv.plot()


dict_keys(['BTCUSDT', 'ETHUSDT', 'XRPUSDT', 'BNBUSDT', 'ADAUSDT', 'LTCUSDT'])
(1031, 5)


FigureWidget({
    'data': [{'close': array([ 6698.5,  6733.5,  6354. , ..., 21118. , 20674. , 21058.5]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([ 6745.5,  6767. ,  6838. , ..., 21626.5, 21673. , 21173.5]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([ 6500. ,  6512. ,  6235. , ..., 20824.5, 20385. , 20645. ]),
              'name': 'OHLC',
              'open': array([ 6500. ,  6698.5,  6733.5, ..., 21179. , 21118. , 20674. ]),
              'type': 'ohlc',
              'uid': 'adf6f900-fc60-4bb6-b011-940c64b01bd4',
              'x': array([datetime.datetime(2020, 3, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 3, 26, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 3, 27, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 1, 17, 0, 0, tzinfo=datetime.timezone.utc),
   

In [51]:
ohlcv = data.concat()
ohlcv.keys()
ohlcv['Open'].shape

(1031, 6)

In [52]:
ohlcv_data={}
for k, v in ohlcv.items():
    ohlcv_data[k] = v.vbt.range_split(range_len=window_len.days, n=window_count)[0]
ohlcv_data['Open']

split_idx      0                                                1            \
symbol     BTCUSDT ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT ETHUSDT   
0           6500.0     NaN     NaN     NaN     NaN     NaN   6733.5     NaN   
1           6698.5     NaN     NaN     NaN     NaN     NaN   6354.0     NaN   
2           6733.5     NaN     NaN     NaN     NaN     NaN   6230.5     NaN   
3           6354.0     NaN     NaN     NaN     NaN     NaN   5873.0     NaN   
4           6230.5     NaN     NaN     NaN     NaN     NaN   6389.5     NaN   
..             ...     ...     ...     ...     ...     ...      ...     ...   
175        10778.5     NaN     NaN     NaN     NaN     NaN  10928.0     NaN   
176        10947.0     NaN     NaN     NaN     NaN     NaN  10917.0     NaN   
177        10928.0     NaN     NaN     NaN     NaN     NaN  11071.0     NaN   
178        10917.0     NaN     NaN     NaN     NaN     NaN  10912.0     NaN   
179        11071.0     NaN     NaN     NaN     NaN     NaN  10408.0     NaN   

split_idx                  ...     398                              399  \
symbol    XRPUSDT BNBUSDT  ... XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT   
0             NaN     NaN  ...  0.3664  265.30  0.4990   58.02  22434.5   
1             NaN     NaN  ...  0.3575  262.20  0.4827   56.06  22573.5   
2             NaN     NaN  ...  0.3590  258.90  0.5167   56.35  21301.0   
3             NaN     NaN  ...  0.3586  261.45  0.5118   58.35  21232.0   
4             NaN     NaN  ...  0.3357  244.35  0.4749   53.93  22938.0   
..            ...     ...  ...     ...     ...     ...     ...      ...   
175           NaN     NaN  ...  0.3747  287.45  0.3291   86.04  20952.5   
176           NaN     NaN  ...  0.3856  293.60  0.3454   86.23  20878.0   
177           NaN     NaN  ...  0.3954  305.20  0.3525   87.89  21179.0   
178           NaN     NaN  ...  0.3847  302.05  0.3508   87.43  21118.0   
179           NaN     NaN  ...  0.3856  298.95  0.3503   85.91  20674.0   

split_idx                                           
symbol     ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  
0          1547.80  0.3590  258.90  0.5167   56.35  
1          1597.20  0.3586  261.45  0.5118   58.35  
2          1440.00  0.3357  244.35  0.4749   53.93  
3          1447.95  0.3368  249.25  0.4675   53.89  
4          1635.10  0.3592  271.55  0.5114   59.02  
..             ...     ...     ...     ...     ...  
175        1549.85  0.3954  305.20  0.3525   87.89  
176        1552.35  0.3847  302.05  0.3508   87.43  
177        1576.50  0.3856  298.95  0.3503   85.91  
178        1564.59  0.3868  298.90  0.3455   87.00  
179        1510.65  0.3781  286.40  0.3264   82.68  

[180 rows x 2400 columns]

In [53]:
ohlcv_indexs = pd.Series(ohlcv['Open'].vbt.range_split(range_len=window_len.days, n=window_count)[1])
ohlcv_indexs

0      DatetimeIndex(['2020-03-25 00:00:00+00:00', '2...
1      DatetimeIndex(['2020-03-27 00:00:00+00:00', '2...
2      DatetimeIndex(['2020-03-29 00:00:00+00:00', '2...
3      DatetimeIndex(['2020-03-31 00:00:00+00:00', '2...
4      DatetimeIndex(['2020-04-03 00:00:00+00:00', '2...
                             ...                        
395    DatetimeIndex(['2022-07-15 00:00:00+00:00', '2...
396    DatetimeIndex(['2022-07-18 00:00:00+00:00', '2...
397    DatetimeIndex(['2022-07-20 00:00:00+00:00', '2...
398    DatetimeIndex(['2022-07-22 00:00:00+00:00', '2...
399    DatetimeIndex(['2022-07-24 00:00:00+00:00', '2...
Length: 400, dtype: object

In [54]:
ohlcv_data['Open'].columns

MultiIndex([(  0, 'BTCUSDT'),
            (  0, 'ETHUSDT'),
            (  0, 'XRPUSDT'),
            (  0, 'BNBUSDT'),
            (  0, 'ADAUSDT'),
            (  0, 'LTCUSDT'),
            (  1, 'BTCUSDT'),
            (  1, 'ETHUSDT'),
            (  1, 'XRPUSDT'),
            (  1, 'BNBUSDT'),
            ...
            (398, 'XRPUSDT'),
            (398, 'BNBUSDT'),
            (398, 'ADAUSDT'),
            (398, 'LTCUSDT'),
            (399, 'BTCUSDT'),
            (399, 'ETHUSDT'),
            (399, 'XRPUSDT'),
            (399, 'BNBUSDT'),
            (399, 'ADAUSDT'),
            (399, 'LTCUSDT')],
           names=['split_idx', 'symbol'], length=2400)

In [55]:
entries = pd.DataFrame.vbt.signals.empty_like(ohlcv_data['Open'])
entries.iloc[0, : ] = True
entries

split_idx     0                                               1            \
symbol    BTCUSDT ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT BTCUSDT ETHUSDT   
0            True    True    True    True    True    True    True    True   
1           False   False   False   False   False   False   False   False   
2           False   False   False   False   False   False   False   False   
3           False   False   False   False   False   False   False   False   
4           False   False   False   False   False   False   False   False   
..            ...     ...     ...     ...     ...     ...     ...     ...   
175         False   False   False   False   False   False   False   False   
176         False   False   False   False   False   False   False   False   
177         False   False   False   False   False   False   False   False   
178         False   False   False   False   False   False   False   False   
179         False   False   False   False   False   False   False   False   

split_idx                  ...     398                             399  \
symbol    XRPUSDT BNBUSDT  ... XRPUSDT BNBUSDT ADAUSDT LTCUSDT BTCUSDT   
0            True    True  ...    True    True    True    True    True   
1           False   False  ...   False   False   False   False   False   
2           False   False  ...   False   False   False   False   False   
3           False   False  ...   False   False   False   False   False   
4           False   False  ...   False   False   False   False   False   
..            ...     ...  ...     ...     ...     ...     ...     ...   
175         False   False  ...   False   False   False   False   False   
176         False   False  ...   False   False   False   False   False   
177         False   False  ...   False   False   False   False   False   
178         False   False  ...   False   False   False   False   False   
179         False   False  ...   False   False   False   False   False   

split_idx                                          
symbol    ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  
0            True    True    True    True    True  
1           False   False   False   False   False  
2           False   False   False   False   False  
3           False   False   False   False   False  
4           False   False   False   False   False  
..            ...     ...     ...     ...     ...  
175         False   False   False   False   False  
176         False   False   False   False   False  
177         False   False   False   False   False  
178         False   False   False   False   False  
179         False   False   False   False   False  

[180 rows x 2400 columns]

https://vectorbt.dev/api/signals/generators/#vectorbt.signals.generators.OHLCSTX

In [89]:
sl_ohlcstx = vbt.OHLCSTX.run(
    entries,
    ohlcv_data["Open"],
    ohlcv_data["High"],
    ohlcv_data["Low"],
    ohlcv_data["Close"],
    sl_stop=list(stops),
)
sl_exits = sl_ohlcstx.exits.copy()
sl_close = sl_ohlcstx.close.copy()
sl_exits.vbt.rename_levels(
    {'ohlcstx_sl_stop': 'stop_value'}, strict=False)
sl_close.vbt.rename_levels(
    {'ohlcstx_sl_stop': 'stop_value'}, strict=False)


stop_value     0.01                                                           \
split_idx       0                                                1             
symbol      BTCUSDT ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT ETHUSDT   
0            6698.5     NaN     NaN     NaN     NaN     NaN   6354.0     NaN   
1            6733.5     NaN     NaN     NaN     NaN     NaN   6230.5     NaN   
2            6354.0     NaN     NaN     NaN     NaN     NaN   5873.0     NaN   
3            6230.5     NaN     NaN     NaN     NaN     NaN   6389.5     NaN   
4            5873.0     NaN     NaN     NaN     NaN     NaN   6405.0     NaN   
..              ...     ...     ...     ...     ...     ...      ...     ...   
175         10947.0     NaN     NaN     NaN     NaN     NaN  10917.0     NaN   
176         10928.0     NaN     NaN     NaN     NaN     NaN  11071.0     NaN   
177         10917.0     NaN     NaN     NaN     NaN     NaN  10912.0     NaN   
178         11071.0     NaN     NaN     NaN     NaN     NaN  10408.0     NaN   
179         10912.0     NaN     NaN     NaN     NaN     NaN  10521.0     NaN   

stop_value                  ...    1.00                                   \
split_idx                   ...     398                              399   
symbol     XRPUSDT BNBUSDT  ... XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT   
0              NaN     NaN  ...  0.3575  262.20  0.4827   56.06  22573.5   
1              NaN     NaN  ...  0.3590  258.90  0.5167   56.35  21301.0   
2              NaN     NaN  ...  0.3586  261.45  0.5118   58.35  21232.0   
3              NaN     NaN  ...  0.3357  244.35  0.4749   53.93  22938.0   
4              NaN     NaN  ...  0.3368  249.25  0.4675   53.89  23839.0   
..             ...     ...  ...     ...     ...     ...     ...      ...   
175            NaN     NaN  ...  0.3856  293.60  0.3454   86.23  20878.0   
176            NaN     NaN  ...  0.3954  305.20  0.3525   87.89  21179.0   
177            NaN     NaN  ...  0.3847  302.05  0.3508   87.43  21118.0   
178            NaN     NaN  ...  0.3856  298.95  0.3503   85.91  20674.0   
179            NaN     NaN  ...  0.3868  298.90  0.3455   87.00  21058.5   

stop_value                                           
split_idx                                            
symbol      ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  
0           1597.20  0.3586  261.45  0.5118   58.35  
1           1440.00  0.3357  244.35  0.4749   53.93  
2           1447.95  0.3368  249.25  0.4675   53.89  
3           1635.10  0.3592  271.55  0.5114   59.02  
4           1723.70  0.3733  277.20  0.5127   63.41  
..              ...     ...     ...     ...     ...  
175         1552.35  0.3847  302.05  0.3508   87.43  
176         1576.50  0.3856  298.95  0.3503   85.91  
177         1564.59  0.3868  298.90  0.3455   87.00  
178         1510.65  0.3781  286.40  0.3264   82.68  
179         1550.38  0.3929  294.30  0.3379   84.15  

[180 rows x 240000 columns]

In [91]:
slt_ohlcstx = vbt.OHLCSTX.run(
    entries,
    ohlcv_data["Open"],
    ohlcv_data["High"],
    ohlcv_data["Low"],
    ohlcv_data["Close"],
    sl_trail=list(stops),
)
slt_exits = slt_ohlcstx.exits.copy()
slt_close = slt_ohlcstx.close.copy()
slt_exits.vbt.rename_levels(
    {'ohlcstx_tptrail': 'stop_value'}, strict=False)
slt_close.vbt.rename_levels(
    {'ohlcstx_tptrail': 'stop_value'}, strict=False)


ohlcstx_sl_trail     0.01                                                   \
split_idx             0                                                1     
symbol            BTCUSDT ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT   
0                  6698.5     NaN     NaN     NaN     NaN     NaN   6354.0   
1                  6733.5     NaN     NaN     NaN     NaN     NaN   6230.5   
2                  6354.0     NaN     NaN     NaN     NaN     NaN   5873.0   
3                  6230.5     NaN     NaN     NaN     NaN     NaN   6389.5   
4                  5873.0     NaN     NaN     NaN     NaN     NaN   6405.0   
..                    ...     ...     ...     ...     ...     ...      ...   
175               10947.0     NaN     NaN     NaN     NaN     NaN  10917.0   
176               10928.0     NaN     NaN     NaN     NaN     NaN  11071.0   
177               10917.0     NaN     NaN     NaN     NaN     NaN  10912.0   
178               11071.0     NaN     NaN     NaN     NaN     NaN  10408.0   
179               10912.0     NaN     NaN     NaN     NaN     NaN  10521.0   

ohlcstx_sl_trail                          ...    1.00                          \
split_idx                                 ...     398                           
symbol           ETHUSDT XRPUSDT BNBUSDT  ... XRPUSDT BNBUSDT ADAUSDT LTCUSDT   
0                    NaN     NaN     NaN  ...  0.3575  262.20  0.4827   56.06   
1                    NaN     NaN     NaN  ...  0.3590  258.90  0.5167   56.35   
2                    NaN     NaN     NaN  ...  0.3586  261.45  0.5118   58.35   
3                    NaN     NaN     NaN  ...  0.3357  244.35  0.4749   53.93   
4                    NaN     NaN     NaN  ...  0.3368  249.25  0.4675   53.89   
..                   ...     ...     ...  ...     ...     ...     ...     ...   
175                  NaN     NaN     NaN  ...  0.3856  293.60  0.3454   86.23   
176                  NaN     NaN     NaN  ...  0.3954  305.20  0.3525   87.89   
177                  NaN     NaN     NaN  ...  0.3847  302.05  0.3508   87.43   
178                  NaN     NaN     NaN  ...  0.3856  298.95  0.3503   85.91   
179                  NaN     NaN     NaN  ...  0.3868  298.90  0.3455   87.00   

ohlcstx_sl_trail                                                    
split_idx             399                                           
symbol            BTCUSDT  ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  
0                 22573.5  1597.20  0.3586  261.45  0.5118   58.35  
1                 21301.0  1440.00  0.3357  244.35  0.4749   53.93  
2                 21232.0  1447.95  0.3368  249.25  0.4675   53.89  
3                 22938.0  1635.10  0.3592  271.55  0.5114   59.02  
4                 23839.0  1723.70  0.3733  277.20  0.5127   63.41  
..                    ...      ...     ...     ...     ...     ...  
175               20878.0  1552.35  0.3847  302.05  0.3508   87.43  
176               21179.0  1576.50  0.3856  298.95  0.3503   85.91  
177               21118.0  1564.59  0.3868  298.90  0.3455   87.00  
178               20674.0  1510.65  0.3781  286.40  0.3264   82.68  
179               21058.5  1550.38  0.3929  294.30  0.3379   84.15  

[180 rows x 240000 columns]

In [92]:
tp_ohlcstx = vbt.OHLCSTX.run(
    entries,
    ohlcv_data["Open"],
    ohlcv_data["High"],
    ohlcv_data["Low"],
    ohlcv_data["Close"],
    tp_stop=list(stops),
)
tp_exits = tp_ohlcstx.exits.copy()
tp_close = tp_ohlcstx.close.copy()
tp_exits.vbt.rename_levels(
    {'ohlcstx_tp_stop': 'stop_value'}, strict=False)
tp_close.vbt.rename_levels(
    {'ohlcstx_tp_stop': 'stop_value'}, strict=False)


stop_value     0.01                                                           \
split_idx       0                                                1             
symbol      BTCUSDT ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT ETHUSDT   
0            6698.5     NaN     NaN     NaN     NaN     NaN   6354.0     NaN   
1            6733.5     NaN     NaN     NaN     NaN     NaN   6230.5     NaN   
2            6354.0     NaN     NaN     NaN     NaN     NaN   5873.0     NaN   
3            6230.5     NaN     NaN     NaN     NaN     NaN   6389.5     NaN   
4            5873.0     NaN     NaN     NaN     NaN     NaN   6405.0     NaN   
..              ...     ...     ...     ...     ...     ...      ...     ...   
175         10947.0     NaN     NaN     NaN     NaN     NaN  10917.0     NaN   
176         10928.0     NaN     NaN     NaN     NaN     NaN  11071.0     NaN   
177         10917.0     NaN     NaN     NaN     NaN     NaN  10912.0     NaN   
178         11071.0     NaN     NaN     NaN     NaN     NaN  10408.0     NaN   
179         10912.0     NaN     NaN     NaN     NaN     NaN  10521.0     NaN   

stop_value                  ...    1.00                                   \
split_idx                   ...     398                              399   
symbol     XRPUSDT BNBUSDT  ... XRPUSDT BNBUSDT ADAUSDT LTCUSDT  BTCUSDT   
0              NaN     NaN  ...  0.3575  262.20  0.4827   56.06  22573.5   
1              NaN     NaN  ...  0.3590  258.90  0.5167   56.35  21301.0   
2              NaN     NaN  ...  0.3586  261.45  0.5118   58.35  21232.0   
3              NaN     NaN  ...  0.3357  244.35  0.4749   53.93  22938.0   
4              NaN     NaN  ...  0.3368  249.25  0.4675   53.89  23839.0   
..             ...     ...  ...     ...     ...     ...     ...      ...   
175            NaN     NaN  ...  0.3856  293.60  0.3454   86.23  20878.0   
176            NaN     NaN  ...  0.3954  305.20  0.3525   87.89  21179.0   
177            NaN     NaN  ...  0.3847  302.05  0.3508   87.43  21118.0   
178            NaN     NaN  ...  0.3856  298.95  0.3503   85.91  20674.0   
179            NaN     NaN  ...  0.3868  298.90  0.3455   87.00  21058.5   

stop_value                                           
split_idx                                            
symbol      ETHUSDT XRPUSDT BNBUSDT ADAUSDT LTCUSDT  
0           1597.20  0.3586  261.45  0.5118   58.35  
1           1440.00  0.3357  244.35  0.4749   53.93  
2           1447.95  0.3368  249.25  0.4675   53.89  
3           1635.10  0.3592  271.55  0.5114   59.02  
4           1723.70  0.3733  277.20  0.5127   63.41  
..              ...     ...     ...     ...     ...  
175         1552.35  0.3847  302.05  0.3508   87.43  
176         1576.50  0.3856  298.95  0.3503   85.91  
177         1564.59  0.3868  298.90  0.3455   87.00  
178         1510.65  0.3781  286.40  0.3264   82.68  
179         1550.38  0.3929  294.30  0.3379   84.15  

[180 rows x 240000 columns]

In [93]:
sl_exits.columns

MultiIndex([(0.01,   0, 'BTCUSDT'),
            (0.01,   0, 'ETHUSDT'),
            (0.01,   0, 'XRPUSDT'),
            (0.01,   0, 'BNBUSDT'),
            (0.01,   0, 'ADAUSDT'),
            (0.01,   0, 'LTCUSDT'),
            (0.01,   1, 'BTCUSDT'),
            (0.01,   1, 'ETHUSDT'),
            (0.01,   1, 'XRPUSDT'),
            (0.01,   1, 'BNBUSDT'),
            ...
            ( 1.0, 398, 'XRPUSDT'),
            ( 1.0, 398, 'BNBUSDT'),
            ( 1.0, 398, 'ADAUSDT'),
            ( 1.0, 398, 'LTCUSDT'),
            ( 1.0, 399, 'BTCUSDT'),
            ( 1.0, 399, 'ETHUSDT'),
            ( 1.0, 399, 'XRPUSDT'),
            ( 1.0, 399, 'BNBUSDT'),
            ( 1.0, 399, 'ADAUSDT'),
            ( 1.0, 399, 'LTCUSDT')],
           names=['ohlcstx_sl_stop', 'split_idx', 'symbol'], length=240000)